# Tutorial: Upload your own Catalog

The purpose of this tutorial is to illustrate how you can construct your own catalog to upload to the FastSpecFit web-app at https://fastspecfit.desi.lbl.gov.

John Moustakas  
Siena College  
November 2023

In [1]:
import os
import fitsio
import numpy as np
from astropy.table import Table

#### Read the data

In [2]:
datadir = '/global/cfs/cdirs/desi/spectro/fastspecfit/fuji/v3.0/catalogs'
%time fast = Table(fitsio.read(os.path.join(datadir, 'fastspec-fuji.fits'), 'FASTSPEC'))
%time meta = Table(fitsio.read(os.path.join(datadir, 'fastspec-fuji.fits'), 'METADATA'))

CPU times: user 14.9 s, sys: 9.2 s, total: 24.1 s
Wall time: 25.2 s
CPU times: user 1.81 s, sys: 1.1 s, total: 2.91 s
Wall time: 3.26 s


In [3]:
print(meta.colnames)

['TARGETID', 'SURVEY', 'PROGRAM', 'HEALPIX', 'TILEID_LIST', 'RA', 'DEC', 'COADD_FIBERSTATUS', 'DESI_TARGET', 'BGS_TARGET', 'MWS_TARGET', 'SCND_TARGET', 'Z', 'ZWARN', 'DELTACHI2', 'SPECTYPE', 'Z_RR', 'TSNR2_BGS', 'TSNR2_LRG', 'TSNR2_ELG', 'TSNR2_QSO', 'TSNR2_LYA', 'PHOTSYS', 'LS_ID', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'FIBERTOTFLUX_G', 'FIBERTOTFLUX_R', 'FIBERTOTFLUX_Z', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FLUX_W1', 'FLUX_W2', 'FLUX_W3', 'FLUX_W4', 'FLUX_IVAR_G', 'FLUX_IVAR_R', 'FLUX_IVAR_Z', 'FLUX_IVAR_W1', 'FLUX_IVAR_W2', 'FLUX_IVAR_W3', 'FLUX_IVAR_W4', 'EBV', 'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z', 'MW_TRANSMISSION_W1', 'MW_TRANSMISSION_W2', 'MW_TRANSMISSION_W3', 'MW_TRANSMISSION_W4', 'CMX_TARGET', 'SV1_DESI_TARGET', 'SV1_BGS_TARGET', 'SV1_MWS_TARGET', 'SV2_DESI_TARGET', 'SV2_BGS_TARGET', 'SV2_MWS_TARGET', 'SV3_DESI_TARGET', 'SV3_BGS_TARGET', 'SV3_MWS_TARGET', 'SV1_SCND_TARGET', 'SV2_SCND_TARGET', 'SV3_SCND_TARGET']


In [4]:
print(fast.colnames)

['TARGETID', 'SURVEY', 'PROGRAM', 'HEALPIX', 'Z', 'COEFF', 'RCHI2', 'RCHI2_CONT', 'RCHI2_PHOT', 'SNR_B', 'SNR_R', 'SNR_Z', 'SMOOTHCORR_B', 'SMOOTHCORR_R', 'SMOOTHCORR_Z', 'VDISP', 'VDISP_IVAR', 'AV', 'AGE', 'ZZSUN', 'LOGMSTAR', 'SFR', 'DN4000', 'DN4000_OBS', 'DN4000_IVAR', 'DN4000_MODEL', 'FLUX_SYNTH_G', 'FLUX_SYNTH_R', 'FLUX_SYNTH_Z', 'FLUX_SYNTH_SPECMODEL_G', 'FLUX_SYNTH_SPECMODEL_R', 'FLUX_SYNTH_SPECMODEL_Z', 'FLUX_SYNTH_PHOTMODEL_G', 'FLUX_SYNTH_PHOTMODEL_R', 'FLUX_SYNTH_PHOTMODEL_Z', 'FLUX_SYNTH_PHOTMODEL_W1', 'FLUX_SYNTH_PHOTMODEL_W2', 'FLUX_SYNTH_PHOTMODEL_W3', 'FLUX_SYNTH_PHOTMODEL_W4', 'ABSMAG10_DECAM_G', 'ABSMAG10_IVAR_DECAM_G', 'KCORR10_DECAM_G', 'ABSMAG10_DECAM_R', 'ABSMAG10_IVAR_DECAM_R', 'KCORR10_DECAM_R', 'ABSMAG10_DECAM_Z', 'ABSMAG10_IVAR_DECAM_Z', 'KCORR10_DECAM_Z', 'ABSMAG00_U', 'ABSMAG00_IVAR_U', 'KCORR00_U', 'ABSMAG00_B', 'ABSMAG00_IVAR_B', 'KCORR00_B', 'ABSMAG00_V', 'ABSMAG00_IVAR_V', 'KCORR00_V', 'ABSMAG01_SDSS_U', 'ABSMAG01_IVAR_SDSS_U', 'KCORR01_SDSS_U', 'ABSMAG

In [5]:
def add_bitnames(meta):
    """Convenience function to add simple (low-level) targeting bits to each object."""

    desi_bitnames = np.zeros(len(meta), dtype='U300')
    bgs_bitnames = np.zeros(len(meta), dtype='U300')
    mws_bitnames = np.zeros(len(meta), dtype='U300')
    scnd_bitnames = np.zeros(len(meta), dtype='U300')
    cmx_bitnames = np.zeros(len(meta), dtype='U300')
    targetclass = np.zeros(len(meta), dtype='U300')

    def get_targetclass(targetclass, name):
        for cc in ['BGS', 'LRG', 'ELG', 'QSO', 'MWS', 'SCND', 'STD']:
            if cc in name:
                for iobj, tclass in enumerate(targetclass):
                    if tclass == '':
                        targetclass[iobj] = cc
                    else:
                        if not cc in tclass: # only once
                            targetclass[iobj] = ' '.join([tclass, cc])
        return targetclass
    
    for survey, prefix in zip(['SV3'], ['SV3_']):
    #for survey, prefix in zip(['CMX', 'SV1', 'SV2', 'SV3', 'MAIN'], ['CMX_', 'SV1_', 'SV2_', 'SV3_', '']):
        I = meta['SURVEY'] == survey.lower()
        if np.sum(I) > 0:
            if survey == 'MAIN':
                from desitarget.targetmask import desi_mask, bgs_mask, mws_mask, scnd_mask
            elif survey == 'SV1':
                from desitarget.sv1.sv1_targetmask import desi_mask, bgs_mask, mws_mask, scnd_mask
            elif survey == 'SV2':
                from desitarget.sv2.sv2_targetmask import desi_mask, bgs_mask, mws_mask, scnd_mask
            elif survey == 'SV3':
                from desitarget.sv3.sv3_targetmask import desi_mask, bgs_mask, mws_mask, scnd_mask
            elif survey == 'CMX':
                from desitarget.cmx.cmx_targetmask import cmx_mask

            if survey == 'CMX':
                for name in cmx_mask.names():
                    J = np.where(meta['CMX_TARGET'.format(prefix)] & cmx_mask.mask(name) != 0)[0]
                    if len(J) > 0:
                        cmx_bitnames[J] = [' '.join([bit, name]) for bit in cmx_bitnames[J]]
                        #if 'QSO' in name:
                        #    pdb.set_trace()
                        #print(name, targetclass[J])
                        targetclass[J] = get_targetclass(targetclass[J], name)
            else:
                for name in desi_mask.names():
                    J = np.where(meta['{}DESI_TARGET'.format(prefix)] & desi_mask.mask(name) != 0)[0]
                    if len(J) > 0:
                        desi_bitnames[J] = [' '.join([bit, name]) for bit in desi_bitnames[J]]
                        targetclass[J] = get_targetclass(targetclass[J], name)
                        
                for name in bgs_mask.names():
                    J = np.where(meta['{}BGS_TARGET'.format(prefix)] & bgs_mask.mask(name) != 0)[0]
                    if len(J) > 0:
                        bgs_bitnames[J] = [' '.join([bit, name]) for bit in bgs_bitnames[J]]
                        targetclass[J] = get_targetclass(targetclass[J], name)
                        
                for name in mws_mask.names():
                    J = np.where(meta['{}MWS_TARGET'.format(prefix)] & mws_mask.mask(name) != 0)[0]
                    if len(J) > 0:
                        mws_bitnames[J] = [' '.join([bit, name]) for bit in mws_bitnames[J]]
                        targetclass[J] = get_targetclass(targetclass[J], name)
                        
                for name in scnd_mask.names():
                    J = np.where(meta['{}SCND_TARGET'.format(prefix)] & scnd_mask.mask(name) != 0)[0]
                    if len(J) > 0:
                        scnd_bitnames[J] = [' '.join([bit, name]) for bit in scnd_bitnames[J]]
                        targetclass[J] = get_targetclass(targetclass[J], name)

    meta['DESI_BITNAMES'] = desi_bitnames
    meta['BGS_BITNAMES'] = bgs_bitnames
    meta['MWS_BITNAMES'] = mws_bitnames
    meta['SCND_BITNAMES'] = scnd_bitnames
    meta['CMX_BITNAMES'] = cmx_bitnames
    meta['TARGETCLASS'] = targetclass

In [6]:
add_bitnames(meta)

#### Select the samples of interest (all from SV3):
1. 5 massive, low-redshift, early-type BGS targets.
2. 5 strong [OII]-emitting ELG targets.
3. 5 high-redshift QSO targets with strong MgII emission.
4. 5 low-redshift broad-line AGN and BGS targets.

In [7]:
I1 = np.where((meta['SURVEY'] == 'sv3') * 
              np.isin(meta['TARGETCLASS'], 'BGS') * ~np.isin(meta['TARGETCLASS'], 'SCND') *
              (fast['Z'] < 0.4) * (fast['DN4000_MODEL'] > 1.8) * 
              (fast['LOGMSTAR'] > 10.5))[0][:5]

In [8]:
I2 = np.where((meta['SURVEY'] == 'sv3') * 
              np.isin(meta['TARGETCLASS'], 'ELG') * ~np.isin(meta['TARGETCLASS'], 'SCND') *
              (fast['Z'] > 0.7) * 
              (fast['OII_3726_AMP']*np.sqrt(fast['OII_3726_AMP_IVAR']) > 3) *
              (fast['OII_3729_AMP']*np.sqrt(fast['OII_3729_AMP_IVAR']) > 3) *
              ((fast['OII_3726_EW']+fast['OII_3729_EW']) > 30))[0][:5]

In [9]:
I3 = np.where((meta['SURVEY'] == 'sv3') * 
              np.isin(meta['TARGETCLASS'], 'QSO') * ~np.isin(meta['TARGETCLASS'], 'SCND') *
              (fast['MGII_2803_AMP']*np.sqrt(fast['MGII_2803_AMP_IVAR']) > 5) *
              (fast['MGII_2803_SIGMA'] > 1000))[0][:5]

In [10]:
I4 = np.where((meta['SURVEY'] == 'sv3') * 
              np.isin(meta['TARGETCLASS'], 'BGS') * ~np.isin(meta['TARGETCLASS'], 'SCND') *
              (fast['Z'] < 0.3) * (fast['HALPHA_BROAD_SIGMA'] > 1000))[0][:5]

In [11]:
I = np.hstack((I1, I2, I3, I4))
mysample = fast[I]
mysample

TARGETID,SURVEY,PROGRAM,HEALPIX,Z,COEFF,RCHI2,RCHI2_CONT,RCHI2_PHOT,SNR_B,SNR_R,SNR_Z,SMOOTHCORR_B,SMOOTHCORR_R,SMOOTHCORR_Z,VDISP,VDISP_IVAR,AV,AGE,ZZSUN,LOGMSTAR,SFR,DN4000,DN4000_OBS,DN4000_IVAR,DN4000_MODEL,FLUX_SYNTH_G,FLUX_SYNTH_R,FLUX_SYNTH_Z,FLUX_SYNTH_SPECMODEL_G,FLUX_SYNTH_SPECMODEL_R,FLUX_SYNTH_SPECMODEL_Z,FLUX_SYNTH_PHOTMODEL_G,FLUX_SYNTH_PHOTMODEL_R,FLUX_SYNTH_PHOTMODEL_Z,FLUX_SYNTH_PHOTMODEL_W1,FLUX_SYNTH_PHOTMODEL_W2,FLUX_SYNTH_PHOTMODEL_W3,FLUX_SYNTH_PHOTMODEL_W4,ABSMAG10_DECAM_G,ABSMAG10_IVAR_DECAM_G,KCORR10_DECAM_G,ABSMAG10_DECAM_R,ABSMAG10_IVAR_DECAM_R,KCORR10_DECAM_R,ABSMAG10_DECAM_Z,ABSMAG10_IVAR_DECAM_Z,KCORR10_DECAM_Z,ABSMAG00_U,ABSMAG00_IVAR_U,KCORR00_U,ABSMAG00_B,ABSMAG00_IVAR_B,KCORR00_B,ABSMAG00_V,ABSMAG00_IVAR_V,KCORR00_V,ABSMAG01_SDSS_U,ABSMAG01_IVAR_SDSS_U,KCORR01_SDSS_U,ABSMAG01_SDSS_G,ABSMAG01_IVAR_SDSS_G,KCORR01_SDSS_G,ABSMAG01_SDSS_R,ABSMAG01_IVAR_SDSS_R,KCORR01_SDSS_R,ABSMAG01_SDSS_I,ABSMAG01_IVAR_SDSS_I,KCORR01_SDSS_I,ABSMAG01_SDSS_Z,ABSMAG01_IVAR_SDSS_Z,KCORR01_SDSS_Z,ABSMAG01_W1,ABSMAG01_IVAR_W1,KCORR01_W1,LOGLNU_1500,LOGLNU_2800,LOGL_1450,LOGL_1700,LOGL_3000,LOGL_5100,FLYA_1215_CONT,FOII_3727_CONT,FHBETA_CONT,FOIII_5007_CONT,FHALPHA_CONT,RCHI2_LINE,DELTA_LINECHI2,DELTA_LINENDOF,APERCORR,APERCORR_G,APERCORR_R,APERCORR_Z,NARROW_Z,NARROW_ZRMS,BROAD_Z,BROAD_ZRMS,UV_Z,UV_ZRMS,NARROW_SIGMA,NARROW_SIGMARMS,BROAD_SIGMA,BROAD_SIGMARMS,UV_SIGMA,UV_SIGMARMS,MGII_DOUBLET_RATIO,OII_DOUBLET_RATIO,SII_DOUBLET_RATIO,LYALPHA_MODELAMP,LYALPHA_AMP,LYALPHA_AMP_IVAR,LYALPHA_FLUX,LYALPHA_FLUX_IVAR,LYALPHA_BOXFLUX,LYALPHA_BOXFLUX_IVAR,LYALPHA_VSHIFT,LYALPHA_SIGMA,LYALPHA_CONT,LYALPHA_CONT_IVAR,LYALPHA_EW,LYALPHA_EW_IVAR,LYALPHA_FLUX_LIMIT,LYALPHA_EW_LIMIT,LYALPHA_CHI2,LYALPHA_NPIX,OI_1304_MODELAMP,OI_1304_AMP,OI_1304_AMP_IVAR,OI_1304_FLUX,OI_1304_FLUX_IVAR,OI_1304_BOXFLUX,OI_1304_BOXFLUX_IVAR,OI_1304_VSHIFT,OI_1304_SIGMA,OI_1304_CONT,OI_1304_CONT_IVAR,OI_1304_EW,OI_1304_EW_IVAR,OI_1304_FLUX_LIMIT,OI_1304_EW_LIMIT,OI_1304_CHI2,OI_1304_NPIX,SILIV_1396_MODELAMP,SILIV_1396_AMP,SILIV_1396_AMP_IVAR,SILIV_1396_FLUX,SILIV_1396_FLUX_IVAR,SILIV_1396_BOXFLUX,SILIV_1396_BOXFLUX_IVAR,SILIV_1396_VSHIFT,SILIV_1396_SIGMA,SILIV_1396_CONT,SILIV_1396_CONT_IVAR,SILIV_1396_EW,SILIV_1396_EW_IVAR,SILIV_1396_FLUX_LIMIT,SILIV_1396_EW_LIMIT,SILIV_1396_CHI2,SILIV_1396_NPIX,CIV_1549_MODELAMP,CIV_1549_AMP,CIV_1549_AMP_IVAR,CIV_1549_FLUX,CIV_1549_FLUX_IVAR,CIV_1549_BOXFLUX,CIV_1549_BOXFLUX_IVAR,CIV_1549_VSHIFT,CIV_1549_SIGMA,CIV_1549_CONT,CIV_1549_CONT_IVAR,CIV_1549_EW,CIV_1549_EW_IVAR,CIV_1549_FLUX_LIMIT,CIV_1549_EW_LIMIT,CIV_1549_CHI2,CIV_1549_NPIX,HEII_1640_MODELAMP,HEII_1640_AMP,HEII_1640_AMP_IVAR,HEII_1640_FLUX,HEII_1640_FLUX_IVAR,HEII_1640_BOXFLUX,HEII_1640_BOXFLUX_IVAR,HEII_1640_VSHIFT,HEII_1640_SIGMA,HEII_1640_CONT,HEII_1640_CONT_IVAR,HEII_1640_EW,HEII_1640_EW_IVAR,HEII_1640_FLUX_LIMIT,HEII_1640_EW_LIMIT,HEII_1640_CHI2,HEII_1640_NPIX,ALIII_1857_MODELAMP,ALIII_1857_AMP,ALIII_1857_AMP_IVAR,ALIII_1857_FLUX,ALIII_1857_FLUX_IVAR,ALIII_1857_BOXFLUX,ALIII_1857_BOXFLUX_IVAR,ALIII_1857_VSHIFT,ALIII_1857_SIGMA,ALIII_1857_CONT,ALIII_1857_CONT_IVAR,ALIII_1857_EW,ALIII_1857_EW_IVAR,ALIII_1857_FLUX_LIMIT,ALIII_1857_EW_LIMIT,ALIII_1857_CHI2,ALIII_1857_NPIX,SILIII_1892_MODELAMP,SILIII_1892_AMP,SILIII_1892_AMP_IVAR,SILIII_1892_FLUX,SILIII_1892_FLUX_IVAR,SILIII_1892_BOXFLUX,SILIII_1892_BOXFLUX_IVAR,SILIII_1892_VSHIFT,SILIII_1892_SIGMA,SILIII_1892_CONT,SILIII_1892_CONT_IVAR,SILIII_1892_EW,SILIII_1892_EW_IVAR,SILIII_1892_FLUX_LIMIT,SILIII_1892_EW_LIMIT,SILIII_1892_CHI2,SILIII_1892_NPIX,CIII_1908_MODELAMP,CIII_1908_AMP,CIII_1908_AMP_IVAR,CIII_1908_FLUX,CIII_1908_FLUX_IVAR,CIII_1908_BOXFLUX,CIII_1908_BOXFLUX_IVAR,CIII_1908_VSHIFT,CIII_1908_SIGMA,CIII_1908_CONT,CIII_1908_CONT_IVAR,CIII_1908_EW,CIII_1908_EW_IVAR,CIII_1908_FLUX_LIMIT,CIII_1908_EW_LIMIT,CIII_1908_CHI2,CIII_1908_NPIX,MGII_2796_MODELAMP,MGII_2796_AMP,MGII_2796_AMP_IVAR,MGII_2796_FLUX,MGII_2796_FLUX_IVAR,MGII_2796_BOXFLUX,MGII_2796_BOXFLUX_IVAR,MGII_2796_VSHIFT,MGII_2796_SIGMA,MGII_2796_CONT,MG

In [12]:
upload = meta[I]['SURVEY', 'PROGRAM', 'HEALPIX', 'TARGETID', 'TARGETCLASS']
upload.write(os.path.join(os.environ.get('PSCRATCH'), 'upload.fits'), overwrite=True)

In [13]:
upload

SURVEY,PROGRAM,HEALPIX,TARGETID,TARGETCLASS
str7,str6,int32,int64,str300
sv3,bright,25915,39627733927463809,BGS
sv3,bright,25915,39627733927465411,BGS
sv3,bright,25915,39627733927465641,BGS
sv3,bright,25913,39627733935853350,BGS
sv3,bright,25913,39627733935854130,BGS
sv3,dark,25926,39627728017689437,ELG
sv3,dark,25915,39627733927465397,ELG
sv3,dark,25915,39627733927465415,ELG
sv3,dark,25915,39627733927465559,ELG
